# Introduction to Linear Algebra with NumPy and PyTorch
by Pierre Nugues


## The Corpus
We create a dictionary with URLs

In [1]:
classics_url = {'iliad': 'http://classics.mit.edu/Homer/iliad.mb.txt',
                'odyssey': 'http://classics.mit.edu/Homer/odyssey.mb.txt',
                'eclogue': 'http://classics.mit.edu/Virgil/eclogue.mb.txt',
                'georgics': 'http://classics.mit.edu/Virgil/georgics.mb.txt',
                'aeneid': 'http://classics.mit.edu/Virgil/aeneid.mb.txt'}


We read the texts from the URLs

In [2]:
import requests

classics = {}
for key in classics_url:
    classics[key] = requests.get(classics_url[key]).text


We remove the license information to keep only the text

In [3]:
text_bounds = {'iliad': (136, -486), 'odyssey': (138, -486),
               'eclogue': (139, -486), 'georgics': (140, -486), 'aeneid': (138, -486)}


In [4]:
for key in classics:
    classics[key] = classics[key][text_bounds[key][0]:text_bounds[key][1]]


In [5]:
classics['iliad'][:50]


'The Iliad\nBy Homer\n\n\nTranslated by Samuel Butler\n\n'

We additionally write the Iliad and the Odyssey in two text files

In [6]:
with open('iliad.txt', 'w') as f_il, open('odyssey.txt', 'w') as f_od:
    f_il.write(classics['iliad'])
    f_od.write(classics['odyssey'])


We store the corpus in a JSON file

In [7]:
import json

with open('classics.json', 'w') as f:
    json.dump(classics, f)


We read it again

In [8]:
with open('classics.json', 'r') as f:
    classics = json.loads(f.read())


## Utilities

In [9]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'


In [10]:
class Text:
    """Text class to hold and process text"""

    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    def __init__(self, text=None):
        """The constructor called when an object
        is created"""

        self.content = text
        self.length = len(text)
        self.letter_counts = {}

    def count_letters(self, lc=True):
        """Function to count the letters of a text"""

        letter_counts = {}
        if lc:
            text = self.content.lower()
        else:
            text = self.content
        for letter in text:
            if letter.lower() in self.alphabet:
                if letter in letter_counts:
                    letter_counts[letter] += 1
                else:
                    letter_counts[letter] = 1
        self.letter_counts = letter_counts
        return letter_counts


## Imports

In [11]:
import math
import random
import numpy as np
import torch


In [12]:
random.seed(4321)
np.random.seed(4321)
torch.manual_seed(4321)


## The Dataset

Let us read Homer's _Iliad_ and _Odyssey_ and Virgil's _Eclogue_, _Georgics_, and _Aeneid_.

In [13]:
titles = ['iliad', 'odyssey', 'eclogue', 'georgics', 'aeneid']
titles


['iliad', 'odyssey', 'eclogue', 'georgics', 'aeneid']

In [14]:
texts = []
for title in titles:
    texts += [classics[title]]


In [15]:
cnt_dicts = []
for text in texts:
    cnt_dicts += [Text(text).count_letters()]


In [16]:
cnt_lists = []
for cnt_dict in cnt_dicts:
    cnt_lists += [list(map(cnt_dict.get, alphabet))]


In [17]:
cnt_lists[0][:3]


[51020, 8941, 11558]

In [18]:
for i, cnt_list in enumerate(cnt_lists):
    print(titles[i], cnt_lists[i][:10])


iliad [51020, 8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151, 1624]
odyssey [37630, 6598, 8580, 20738, 59783, 10449, 9803, 34787, 28793, 424]
eclogue [2716, 578, 723, 1440, 4366, 846, 808, 2509, 2252, 22]
georgics [6841, 1619, 2017, 4027, 12112, 2424, 2150, 6988, 6038, 59]
aeneid [36678, 6869, 10023, 23866, 55372, 11618, 9607, 33057, 30579, 908]


## Vectors
### NumPy

In [19]:
np.array([2, 3])
np.array([1, 2, 3])


array([1, 2, 3])

Vectors of letter counts

In [20]:
iliad_cnt = np.array(cnt_lists[0])
odyssey_cnt = np.array(cnt_lists[1])
eclogue_cnt = np.array(cnt_lists[2])
georgics_cnt = np.array(cnt_lists[3])
aeneid_cnt = np.array(cnt_lists[4])


In [21]:
iliad_cnt


array([51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,
        1624,  4413, 25311, 16648, 42194, 51270,  9104,   283, 36457,
       41243, 54177, 18409,  6060, 15665,   597, 11908,   284])

In [22]:
odyssey_cnt


array([37630,  6598,  8580, 20738, 59783, 10449,  9803, 34787, 28793,
         424,  3631, 18951, 13060, 31889, 38778,  6679,   256, 25668,
       31352, 40483, 15406,  4803, 12989,   350, 10974,   124])

### The datatype

In [23]:
odyssey_cnt.dtype


dtype('int64')

In [24]:
vector = np.array([1, 2, 3], dtype='int32')
vector


array([1, 2, 3], dtype=int32)

In [25]:
vector.dtype


dtype('int32')

In [26]:
vector = np.array([1, 2, 3], dtype='float64')
vector


array([1., 2., 3.])

In [27]:
np.array([0, 1, 2, 3], dtype='bool')


array([False,  True,  True,  True])

### The vector size

In [28]:
odyssey_cnt.shape


(26,)

### Indices and Slices

In [29]:
vector = np.array([1, 2, 3, 4])
vector[1]   # 2
vector[:1]  # array([1])
vector[1:3]  # array([2, 3])


array([2, 3])

### Operations

In [30]:
np.array([1, 2, 3]) + np.array([4, 5, 6])


array([5, 7, 9])

In [31]:
3 * np.array([1, 2, 3])


array([3, 6, 9])

In [32]:
iliad_cnt + odyssey_cnt      # array([88650,  15539,  20138, ...])


array([ 88650,  15539,  20138,  49071, 137249,  26563,  22398,  84981,
        66944,   2048,   8044,  44262,  29708,  74083,  90048,  15783,
          539,  62125,  72595,  94660,  33815,  10863,  28654,    947,
        22882,    408])

In [33]:
iliad_cnt - odyssey_cnt      # array([13390,  2343,  2978, ...])


array([13390,  2343,  2978,  7595, 17683,  5665,  2792, 15407,  9358,
        1200,   782,  6360,  3588, 10305, 12492,  2425,    27, 10789,
        9891, 13694,  3003,  1257,  2676,   247,   934,   160])

In [34]:

iliad_cnt - 2 * odyssey_cnt  # array([-24240,  -4255,  ...])


array([-24240,  -4255,  -5602, -13143, -42100,  -4784,  -7011, -19380,
       -19435,    776,  -2849, -12591,  -9472, -21584, -26286,  -4254,
         -229, -14879, -21461, -26789, -12403,  -3546, -10313,   -103,
       -10040,     36])

### Comparison with lists

In [35]:
[1, 2, 3] + [4, 5, 6]


[1, 2, 3, 4, 5, 6]

In [36]:
3 * [1, 2, 3]


[1, 2, 3, 1, 2, 3, 1, 2, 3]

### PyTorch
#### Tensors

In [37]:
torch.tensor([2, 3])
torch.tensor([1, 2, 3])


tensor([1, 2, 3])

In [38]:
iliad_cnt_pt = torch.tensor(cnt_lists[0])


In [39]:
iliad_cnt_pt


tensor([51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,  1624,
         4413, 25311, 16648, 42194, 51270,  9104,   283, 36457, 41243, 54177,
        18409,  6060, 15665,   597, 11908,   284])

#### Types

In [40]:
torch.tensor([1, 2, 3]).dtype


torch.int64

In [41]:
torch.tensor([1, 2, 3], dtype=torch.float64)


tensor([1., 2., 3.], dtype=torch.float64)

In [42]:
torch.tensor([0, 1, 2, 3], dtype=torch.bool)


tensor([False,  True,  True,  True])

#### Size

In [43]:
iliad_cnt_pt.size()


torch.Size([26])

#### NumPy/PyTorch Conversion

In [44]:
np_array = np.array([1, 2, 3])
tensor = torch.from_numpy(np_array)
tensor


tensor([1, 2, 3])

In [45]:
tensor = torch.tensor([1, 2, 3])
np_array = tensor.numpy()
np_array


array([1, 2, 3])

#### Device

In [46]:
torch.cuda.is_available()


False

In [47]:
torch.backends.mps.is_available()


False

In [48]:
torch.device('cpu')


device(type='cpu')

In [49]:
torch.device('mps')


device(type='mps')

In [50]:
tensor = torch.tensor([1, 2, 3])
tensor.device


device(type='cpu')

In [51]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


In [52]:
device


device(type='cpu')

In [53]:
tensor = torch.tensor([1, 2, 3], device=device)
tensor


tensor([1, 2, 3])

In [54]:
tensor = torch.tensor([1, 2, 3])
tensor.to(device)


tensor([1, 2, 3])

## NumPy Functions

In [55]:
np.set_printoptions(precision=3)


In [56]:
np.sqrt(iliad_cnt)


array([225.876,  94.557, 107.508, 168.324, 278.327, 126.941, 112.227,
       224.04 , 195.323,  40.299,  66.43 , 159.094, 129.027, 205.412,
       226.429,  95.415,  16.823, 190.937, 203.084, 232.76 , 135.68 ,
        77.846, 125.16 ,  24.434, 109.124,  16.852])

In [57]:
np.cos(iliad_cnt)


array([ 0.86 ,  1.   , -0.997, -0.52 ,  0.821, -0.717, -0.938, -0.715,
        0.877, -0.979, -0.592, -0.688, -0.765, -0.745,  0.702,  0.944,
        0.967, -0.378,  0.985, -0.973,  0.743, -0.991,  0.524,  0.995,
        0.205,  0.309])

In [59]:
np.sqrt(iliad_cnt)


array([225.876,  94.557, 107.508, 168.324, 278.327, 126.941, 112.227,
       224.04 , 195.323,  40.299,  66.43 , 159.094, 129.027, 205.412,
       226.429,  95.415,  16.823, 190.937, 203.084, 232.76 , 135.68 ,
        77.846, 125.16 ,  24.434, 109.124,  16.852])

In [ ]:
np_sqrt = np.vectorize(math.sqrt)
np_sqrt(iliad_cnt)


In [ ]:
np.sum(odyssey_cnt)


In [ ]:
iliad_dist = iliad_cnt / np.sum(iliad_cnt)
odyssey_dist = odyssey_cnt / np.sum(odyssey_cnt)


In [ ]:
iliad_dist


In [ ]:
odyssey_dist


PyTorch

In [ ]:
torch.sqrt(iliad_cnt_pt)


In [ ]:
torch.sum(iliad_cnt_pt)


## Dot Product


In [ ]:
np.dot(iliad_dist, odyssey_dist)


In [ ]:
iliad_dist @ odyssey_dist


In [ ]:
torch.dot(torch.tensor(iliad_dist), torch.tensor(odyssey_dist))


### Norm

In [ ]:
np.linalg.norm([1.0, 2.0, 3.0])


In [ ]:
torch.norm(torch.tensor([1.0, 2.0, 3.0]))


#### Cosine

Finally, we compute the cosine 
$$
\frac{\mathbf{x} \cdot \mathbf{y}}{||\mathbf{x}|| . ||\mathbf{y}||}.
$$

In [ ]:
(iliad_dist @ odyssey_dist) / (
    np.linalg.norm(iliad_dist) *
    np.linalg.norm(odyssey_dist))


## Matrices

### NumPy

We create a matrix from the list of lists

In [ ]:
hv_cnts = np.array(cnt_lists)
hv_cnts


The size

In [ ]:
hv_cnts.shape


The data type

In [ ]:
hv_cnts.dtype


### Indices and Slices

In [ ]:
iliad_cnt[2]


In [ ]:
hv_cnts[1, 2]


Slices

In [ ]:
hv_cnts[1, :]


In [ ]:
hv_cnts[1, :2]


In [ ]:
hv_cnts[3, 2:4]


In [ ]:
hv_cnts[3:, 2:4]


In [ ]:
hv_cnts[:, 2]


### Number of indices

In [ ]:
odyssey_cnt.ndim


In [ ]:
hv_cnts.ndim


## Addition and multiplication by a scalar

In [ ]:
hv_cnts - 2 * hv_cnts


### PyTorch

In [ ]:
hv_cnts_pt = torch.tensor(cnt_lists)
hv_cnts_pt


In [ ]:
hv_cnts_pt.dtype


In [ ]:
hv_cnts_pt.size()


### NumPy Functions

In [ ]:
np.set_printoptions(precision=3)


In [ ]:
np.cos(hv_cnts)


In [ ]:
np.sum(hv_cnts)


In [ ]:
np.sum(hv_cnts, axis=0)


In [ ]:
np.sum(hv_cnts, axis=1)


### PyTorch

In [ ]:
torch.sum(hv_cnts_pt)


In [ ]:
torch.sum(hv_cnts_pt, dim=0)  # array([ 134885,  24605,  32901, ...])


In [ ]:
torch.sum(hv_cnts_pt, dim=1)  # array([630019, 472978,  36332,  96758, 470034])


### Transposing and Reshaping Arrays

In [ ]:
hv_cnts.T


In [ ]:
iliad_cnt.T


In [ ]:
np.array([iliad_cnt])


In [ ]:
np.array([iliad_cnt]).shape


In [ ]:
np.array([iliad_cnt]).T


In [ ]:
iliad_cnt.reshape(1, -1)


In [ ]:
iliad_cnt.reshape(-1, 1)


In [ ]:
torch.unsqueeze(torch.tensor([1, 2, 3]), 1)


### Broadcasting

Relative frequencies of the letter counts

In [ ]:
iliad_dist = (1/np.sum(iliad_cnt)) * iliad_cnt
odyssey_dist = (1/np.sum(odyssey_cnt)) * odyssey_cnt


In [ ]:
iliad_cnt / np.sum(iliad_cnt)


In [ ]:
odyssey_cnt / np.sum(odyssey_cnt)


We can apply an elementwise multiplication or division

In [ ]:
np.array([np.sum(hv_cnts, axis=1)]).T


In [ ]:
hv_dist = hv_cnts / np.array([np.sum(hv_cnts, axis=1)]).T
hv_dist


The Hadamard product

In [ ]:
hv_dist * hv_dist


## Matrix Products

### Matrix-Vector Multiplication

The product of a matrix  ${X}$ by a vector $\mathbf{y}$, ${X}\mathbf{y}$, is a sequence of dot products between the matrix rows and the vector resulting in a column vector:
$$
{X}\mathbf{y} = 
\begin{bmatrix*}
{X}_{1 .} \cdot \mathbf{y} \\
{X}_{2 .} \cdot \mathbf{y} \\
...\\
{X}_{n .} \cdot \mathbf{y} \\
\end{bmatrix*},
$$
where ${X}_{i .}$ denotes the $i^\text{th}$ row of matrix ${X}$. If ${X}$ consists of only one row, we have a matrix product of a row vector by a column vector, which is equivalent to a dot product:
$$
\mathbf{x} \cdot \mathbf{y} =
\begin{bmatrix*}
x_1,&x_2,& ...& x_n\\
\end{bmatrix*}
\begin{bmatrix*}
y_1\\
y_2\\ 
...\\
y_n\\
\end{bmatrix*}
= \sum_{i = 1}^n x_i y_i.
$$

In [ ]:
hv_dist[0, :].reshape(1, -1) @ hv_dist[1, :]


In [ ]:
np.dot(hv_dist[0, :], hv_dist[1, :])


In [ ]:
hv_dist[0, :] @ hv_dist[1, :]


### Document Cosines

We will now compute the cosine of all the pairs of vectors representing the works in the `hv_dist` matrix, _i.e._ the rows of the matrix. For this, we will first compute the dot products of all the pairs, $\mathbf{u} \cdot \mathbf{v}$, then the norms $||\mathbf{u}||$ and  $||\mathbf{v}||$, the products of the norms, $||\mathbf{u}|| \cdot||\mathbf{v}||$, and finally the cosines, $\displaystyle{\frac{\mathbf{u} \cdot \mathbf{v}}{||\mathbf{u}|| \cdot||\mathbf{v}||}}$.

The dot product, $\mathbf{u} \cdot \mathbf{v}$, of all the rows of a matrix $\mathbf{X}$ is simply $\mathbf{X} \mathbf{X}^\intercal$:

In [ ]:
hv_dot = hv_dist @ hv_dist.T
hv_dot


For the vector noms, $||\mathbf{u}||$ and  $||\mathbf{v}||$, we can use `np.linalg.norm()`. Here we will break down the computation with elementary operations. We will apply the Hadamard product to have the square of the coordinates, then sum along the rows, and finally extract the square root:

In [ ]:
hv_norm = np.sqrt(np.sum(hv_dist * hv_dist, axis=1))
hv_norm


We compute the product of the norms, $||\mathbf{u}|| \cdot||\mathbf{v}||$, as a matrix product of a column vector by a row vector as with:
$$
\begin{bmatrix*}
x_1\\
x_2\\
 ...\\
 x_n\\
\end{bmatrix*}
\begin{bmatrix*}
y_1, y_2, ..., y_n\\
\end{bmatrix*}
=
\begin{bmatrix*}
x_1 y_1& x_1 y_2&...&x_1 y_n\\
x_2 y_1& x_2 y_1&...&x_2 y_n\\
 ...\\
 x_ny_1& x_n y_2&...&x_n y_n \\
\end{bmatrix*}. 
$$


In [ ]:
hv_norm_pairs = hv_norm.reshape(-1, 1) @ hv_norm.reshape(1, -1)
hv_norm_pairs


We now nearly done with the cosine. We only need to divide the matrix elements by the norm products, $\displaystyle{\frac{\mathbf{u} \cdot \mathbf{v}}{||\mathbf{u}|| \cdot||\mathbf{v}||}}$

In [ ]:
hv_cos = hv_dot / hv_norm_pairs
hv_cos


## Elementary Mathematical Background for Matrices

In [ ]:
A = np.array([[1, 2],
              [3, 4]])
A @ np.array([5, 6])


### Matrices and Rotations

To finish this notebook, we will have a look at vector rotation. From algebra courses, we know that we can use a matrix to compute a rotation of angle $\theta$. For a two-dimensional vector, the rotation matrix is:
$$
{R}_{\theta} =
\begin{bmatrix*}
\cos \theta &-\sin \theta \\
\sin \theta & \cos \theta \\
\end{bmatrix*}.
$$

In [ ]:
theta_45 = np.pi/4
rot_mat_45 = np.array([[np.cos(theta_45), -np.sin(theta_45)],
                       [np.sin(theta_45), np.cos(theta_45)]])
rot_mat_45


we rotate vector (1, 1) by this angle

In [ ]:
rot_mat_45 @ np.array([1, 1])


The matrix of a sequence of rotations, for instance a rotation of $\pi/6$ followed by a rotation of $\pi/4$, is simply the matrix product of the individual rotations ${R}_{{\theta}_1} {R}_{{\theta}_2}  = {R}_{{\theta}_1 + {\theta}_2}$, here ${R}_{\pi/4} {R}_{\pi/6}  = {R}_{5\pi/12}$. 

In [ ]:
theta_30 = np.pi/6
rot_mat_30 = np.array([[np.cos(theta_30), -np.sin(theta_30)],
                       [np.sin(theta_30), np.cos(theta_30)]])
rot_mat_30


rot_mat_30 @ rot_mat_45


In [ ]:
rot_mat_45 @ rot_mat_30


In [ ]:
np.arccos(0.25881905)


In [ ]:
np.pi/4 + np.pi/6


### Inverting a Matrix

In [ ]:
np.linalg.inv(rot_mat_30)


In [ ]:
np.linalg.inv(rot_mat_30) @ rot_mat_30


In [ ]:
torch.inverse(torch.from_numpy(rot_mat_30))


In [ ]:
torch.inverse(torch.from_numpy(rot_mat_30)) @ torch.from_numpy(rot_mat_30)


## Application to Neural Netwoks

### PyTorch

In [ ]:
layer1 = torch.nn.Linear(3, 4, bias=False)


In [ ]:
layer1.weight


In [ ]:
x = torch.tensor([1.0, 2.0, 3.0])


In [ ]:
layer1(x)


In [ ]:
layer1.weight @ x


Or see: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [ ]:
x @ layer1.weight.T


### More Layers

In [ ]:
layer1 = torch.nn.Linear(3, 4, bias=False)
layer2 = torch.nn.Linear(4, 2, bias=False)
layer3 = torch.nn.Linear(2, 1, bias=False)


In [ ]:
(layer1.weight, layer2.weight, layer3.weight)


In [ ]:
layer3(layer2(layer1(x)))


In [ ]:
x @ layer1.weight.T @ layer2.weight.T @ layer3.weight.T


## Automatic Differentiation

A 3D curve:
$$
z = x^2 + xy + y^2
$$
Its gradient:
$$
\begin{array}{lcl}
\frac{\partial z}{\partial x} &=& 2x + y\\
\frac{\partial z}{\partial y} &=& x + 2y\\
\end{array}
$$

In [ ]:
def f(x, y):
    return x**2 + x * y + y**2


In [ ]:
xt = torch.tensor(3.0, requires_grad=True)
yt = torch.tensor(4.0, requires_grad=True)


In [ ]:
zt = f(xt, yt)
zt


In [ ]:
zt.backward()


In [ ]:
zt


In [ ]:
zt.grad_fn


In [ ]:
zt.grad_fn.next_functions


In [ ]:
zt.grad_fn.next_functions[0][0].next_functions


In [ ]:
zt.grad_fn.next_functions[0][0].next_functions[0][0].next_functions


In [ ]:
zt.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].next_functions


In [ ]:
xt.grad, yt.grad
